In [1]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.callbacks import TensorBoard
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

2025-01-28 20:17:18.031628: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-28 20:17:18.136307: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-28 20:17:18.176684: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-28 20:17:18.188581: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-28 20:17:18.263542: I tensorflow/core/platform/cpu_feature_guar

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [2]:
class Model(keras.Model):
    def __init__(self, norm=False):
        super(Model, self).__init__()
        self.flatten = keras.layers.Flatten()
        self.dense1 = keras.layers.Dense(100, activation="tanh")
        self.dense2 = keras.layers.Dense(100, activation="tanh")
        self.output_layer = keras.layers.Dense(10, activation="softmax")
        self.batchnorm = keras.layers.BatchNormalization()
        self.callbacks = []
        self.norm = norm

    def call(self, x):
        x = self.flatten(x)
        x = self.dense1(x)
        if self.norm:
            x = self.batchnorm(x)
        x = self.dense2(x)
        if self.norm:
            x = self.batchnorm(x)
        return self.output_layer(x)

    def train_loop(
        self, x_train, y_train, x_test, y_test, run_name, batch_size=32, epochs=20, loss="sparse_categorical_crossentropy"
    ):
        self.callbacks.append(TensorBoard(log_dir=f"./runs/{run_name}", histogram_freq=1))
        self.compile(
            optimizer="adam",
            loss=loss,
            metrics=["accuracy"],
        )
        self.fit(
            x_train,
            y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_test, y_test),
            callbacks=self.callbacks,
        )

In [3]:
l = [64, 128, 256, 512, 1024]
for i in l:
    print(f"{i}, no norm, glorot")
    model = Model(norm=False)
    model.dense1 = keras.layers.Dense(i, activation="tanh", kernel_initializer="glorot_normal")
    model.dense2 = keras.layers.Dense(i, activation="tanh", kernel_initializer="glorot_normal")
    model.train_loop(x_train, y_train, x_test, y_test, f"{i}-noNorm-xavier", i, 5)
    print(f"{i}, glorot norm")
    model = Model(norm=True)
    model.dense1 = keras.layers.Dense(i, activation="tanh", kernel_initializer="glorot_normal")
    model.dense2 = keras.layers.Dense(i, activation="tanh", kernel_initializer="glorot_normal")
    model.train_loop(x_train, y_train, x_test, y_test, f"{i}-Norm-xavier", i, 5)


model = Model(norm=False)
model.train_loop(x_train, y_train, x_test, y_test, "noNorm-randomUniform", 128, 5)
model = Model(norm=True)
model.train_loop(x_train, y_train, x_test, y_test, "Norm-randomUniform", 128, 5)

64, no norm, glorot
Epoch 1/5


I0000 00:00:1738075661.719971   23225 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-01-28 20:17:41.747781: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/shusrith/projects/torch/lib/python3.12/site-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'model', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not 

938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7043 - loss: 0.9420 - val_accuracy: 0.8552 - val_loss: 0.4792
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8525 - loss: 0.4692 - val_accuracy: 0.8808 - val_loss: 0.4057
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8709 - loss: 0.4192 - val_accuracy: 0.8669 - val_loss: 0.4204
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8685 - loss: 0.4181 - val_accuracy: 0.8824 - val_loss: 0.3844
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8801 - loss: 0.3760 - val_accuracy: 0.8933 - val_loss: 0.3415
64, glorot norm
Epoch 1/5


/home/shusrith/projects/torch/lib/python3.12/site-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'model_1', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7194 - loss: 0.9159 - val_accuracy: 0.8493 - val_loss: 0.4699
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8526 - loss: 0.4669 - val_accuracy: 0.8607 - val_loss: 0.4437
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8632 - loss: 0.4321 - val_accuracy: 0.8680 - val_loss: 0.4213
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8683 - loss: 0.4090 - val_accuracy: 0.8789 - val_loss: 0.3843
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8822 - loss: 0.3708 - val_accuracy: 0.8967 - val_loss: 0.3408
128, no norm, glorot
Epoch 1/5


...

59/59 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy: 0.7726 - loss: 0.7410 - val_accuracy: 0.9370 - val_loss: 0.2205
Epoch 2/5
59/59 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy: 0.9421 - loss: 0.1981 - val_accuracy: 0.9456 - val_loss: 0.1771
Epoch 3/5
59/59 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy: 0.9516 - loss: 0.1629 - val_accuracy: 0.9506 - val_loss: 0.1655
Epoch 4/5
59/59 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy: 0.9546 - loss: 0.1527 - val_accuracy: 0.9524 - val_loss: 0.1517
Epoch 5/5
59/59 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - accuracy: 0.9569 - loss: 0.1413 - val_accuracy: 0.9564 - val_loss: 0.1476
1024, glorot norm
Epoch 1/5


/home/shusrith/projects/torch/lib/python3.12/site-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'model_9', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


59/59 ━━━━━━━━━━━━━━━━━━━━ 4s 61ms/step - accuracy: 0.7509 - loss: 0.8137 - val_accuracy: 0.9310 - val_loss: 0.2250
Epoch 2/5
59/59 ━━━━━━━━━━━━━━━━━━━━ 4s 61ms/step - accuracy: 0.9435 - loss: 0.1981 - val_accuracy: 0.9454 - val_loss: 0.1806
Epoch 3/5
59/59 ━━━━━━━━━━━━━━━━━━━━ 4s 60ms/step - accuracy: 0.9507 - loss: 0.1673 - val_accuracy: 0.9508 - val_loss: 0.1606
Epoch 4/5
59/59 ━━━━━━━━━━━━━━━━━━━━ 4s 61ms/step - accuracy: 0.9536 - loss: 0.1544 - val_accuracy: 0.9573 - val_loss: 0.1424
Epoch 5/5
59/59 ━━━━━━━━━━━━━━━━━━━━ 4s 63ms/step - accuracy: 0.9594 - loss: 0.1354 - val_accuracy: 0.9552 - val_loss: 0.1408
Epoch 1/5


/home/shusrith/projects/torch/lib/python3.12/site-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'model_10', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7213 - loss: 0.8918 - val_accuracy: 0.8795 - val_loss: 0.3937
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8755 - loss: 0.4021 - val_accuracy: 0.9020 - val_loss: 0.3184
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8918 - loss: 0.3482 - val_accuracy: 0.9079 - val_loss: 0.3048
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9029 - loss: 0.3160 - val_accuracy: 0.9104 - val_loss: 0.2821
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9083 - loss: 0.2909 - val_accuracy: 0.9090 - val_loss: 0.2928
Epoch 1/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7254 - loss: 0.8830 - val_accuracy: 0.8706 - val_loss: 0.4163
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8803 - loss: 0.3844 - val_accuracy: 0.8930 - val_loss: 0.3485
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8773 - loss: 0.3789 - val_accuracy: 0.8862 - val_loss: 0